 ![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# Fine Tuning Model: LoRA Inference

The following code is for the inference part of the LoRA model. The model training can be found in the notebook "LoRA.ipynb" in section 4. Fine Tuning Models. 

### Import Packages and Install Diffusion libraries

In [ ]:
import os
import shutil

In [ ]:
# Diffuser libraries 

!pip install -qq "ipywidgets>=7,<8"
!git clone https://github.com/huggingface/diffusers
!pip install ./diffusers

In [ ]:
%cd /kaggle/working/diffusers/examples/text_to_image
!pip install -r requirements.txt
!pip install bitsandbytes
!pip install transformers gradio ftfy accelerate
!pip install xformers

In [ ]:
# Model Training 
from diffusers import DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch

!pip install huggingface_hub

Replace "TOKEN" with own personal Hugging Face token in order to save model in a private model and dataset. 

In [ ]:
from huggingface_hub import login
login(token="TOKEN") 

For instructions on how to set up the secret token on Kaggle: https://github.com/maelysjb/Comics-GenAI/blob/main/README.md#:~:text=.gitignore-,README,-.md

In [ ]:
from kaggle_secrets import UserSecretsClient
secret_label = "HUGGINGFACE_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)

### Loading the trained model 
Replace the base model path with the desired Hugging Face repository

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

# Define the base Hugging Face repository path
base_model_path = 'nataliabeltran/LoRA600' 

## Pipeline 
pipeline = AutoPipelineForText2Image.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16).to("cuda")
pipeline.load_lora_weights(base_model_path, weight_name="pytorch_lora_weights.safetensors")


### Inference function

In [ ]:
from IPython.display import display
def inference(prompt, num_samples, negative_prompt, guidance_scale,
              num_inference_steps, height, width):
    images = pipeline(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale
    ).images
    for i, image in enumerate(images):
        image.save(f"generated_image_{i}.png")  
        print(f"Generated image {i}:")
        display(image)  

### Display outputs from LoRA model
To generate different images of the character/data change the prompt ensuring to keep the same phrasing that was used while training. 

More images at different training steps for the different models can be found here: https://github.com/maelysjb/Comics-GenAI/tree/main/6.%20Generated%20Images

In [ ]:
# Image Display
from IPython.display import display  
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
prompt = "An image of UnicornGirl in unicorn onesie playing tennis"
num_samples = 5
negative_prompt = ""
guidance_scale = 7.5
num_inference_steps = 50
height = 512
width = 512

inference(prompt, num_samples, negative_prompt, guidance_scale, num_inference_steps, height, width)